In [1]:
# Collaborative Filtering Item Item Similarity

In [2]:
# This is an example Excercise on Memory Based Collaborative Filtering where we use a Movie Preference data from MovieLens
# We show how recommendations are made using an Item-Item similarity matrix

In [3]:
# Created by :

In [4]:
import pandas as pd
import numpy as np

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
ratings = pd.read_csv('ratings.csv')

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
movies = pd.read_csv('movies.csv')

In [9]:
# Identify Genres of each movie and creating a genre vector

In [10]:
def ReturnGenres(genreSeries):
    genreInd = 0
    for genres in genreSeries :
        genreList = genres.split('|')
        for genre in genreList :
            yield genre,genreInd
        genreInd += 1

In [11]:
genres = ReturnGenres(movies.genres)

In [12]:
genreFeature = []
for x in genres :
    genreFeature.append(x[0])
genreFeatureKeys = list(set(genreFeature))

In [13]:
genres = ReturnGenres(movies.genres)

In [14]:
def ReturnGenreFeatures(genreSeries):
    genreFeature = []
    for genres in genreSeries :
        genreList = genres.split('|')
        genreFeature.append(genres)
    return genreFeature

In [15]:
genreFeatures = ReturnGenreFeatures(movies.genres)

In [16]:
genreFeatureVector = []
for genres in genreFeatures :
    temp = [1 if(x in genres) else 0 for x in genreFeatureKeys ]
    genreFeatureVector.append(temp)    

In [17]:
# Creating a Test Set of Movies for Evaluation of our algorithm

In [18]:
from sklearn.cross_validation import train_test_split

/usr/local/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
trainDF, tempDF = train_test_split(ratings, test_size=0.2, random_state=100)
testDF = tempDF.copy()
tempDF.rating = np.nan

/usr/local/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [20]:
testDF = testDF.dropna()

In [21]:
ratings = pd.concat([trainDF, tempDF]).reset_index()

In [22]:
ratingPivot = ratings.pivot(index='movieId', columns='userId', values='rating')

In [23]:
ratingPivot = ratingPivot.reset_index()

In [24]:
tempMovieIndDF = pd.DataFrame()
tempMovieIndDF['Index'] = ratingPivot.index
tempMovieIndDF['movieId'] = ratingPivot.movieId

In [25]:
testDF = testDF.merge(tempMovieIndDF, on='movieId')

In [26]:
ratingPivot = ratingPivot.iloc[:,range(1,ratingPivot.shape[1])]

In [27]:
def CalculateSimilarityWithMissing(v1, v2):
    if (len(v1) != len(v2)) :
        return None
    
    sumV1 = 0
    sumV2 = 0
    dot = 0
    
    v1Nan = np.isnan(v1)
    v2Nan = np.isnan(v2)
    v1[(v1Nan | v2Nan)] = 0
    v2[(v1Nan | v2Nan)] = 0
    
    sumV1 = np.sqrt(np.sum(v1**2))
    sumV2 = np.sqrt(np.sum(v1**2))
    if((sumV1 == 0) or (sumV2 == 0)):
        return 0
    
    dot = np.dot(v1, v2) / (sumV1 * sumV2) 
    
    #for i,x in enumerate(v1) :
    #    if(not(np.isnan(x)) and not(np.isnan(v2[i]))):
    #        dot = dot + x*v2[i]
    #        sumV1 = sumV1 + x**2
    #        sumV2 = sumV2 + v2[i]**2
    #sumV1 = np.sqrt(sumV1)
    #sumV2 = np.sqrt(sumV2)
    
    return dot

In [28]:
ratingPivot.shape

(9066, 671)

In [29]:
ratingPivotMat = ratingPivot.as_matrix()

In [30]:
itemItemSimilarity = np.zeros((ratingPivot.shape[0], ratingPivot.shape[0]))
for i in range(itemItemSimilarity.shape[0]):
    itemItemSimilarity[i,i] = 1

In [31]:
import time

In [32]:
time.ctime()

'Sat Nov 11 04:05:37 2017'

In [33]:
for i in range(itemItemSimilarity.shape[0]) :
    for j in range(0,i) :
        itemItemSimilarity[i,j] = CalculateSimilarityWithMissing(ratingPivotMat[i,:], ratingPivotMat[j,:])
    if((i%500) == 0):
        print i

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000


In [34]:
time.ctime()

'Sat Nov 11 04:25:25 2017'

In [35]:
for i in range(itemItemSimilarity.shape[0]) :
    for j in range(0,i) :
        itemItemSimilarity[j,i] = itemItemSimilarity[i,j] #CalculateSimilarityWithMissing(ratingPivotMat[i,:], ratingPivotMat[j,:])

In [36]:
#Generating Recommendations

In [38]:
users = np.transpose(ratingPivotMat)
recommends = np.dot(users,itemItemSimilarity) / tempItems

NameError: name 'tempItems' is not defined

In [142]:
recommends.shape

(671L, 9066L)

In [37]:
tempUsers = users.copy()
tempUsers[tempUsers > 0] = 1

NameError: name 'users' is not defined

In [140]:
tempItems.shape

(671L, 9066L)

In [139]:
tempItems = np.dot(tempUsers, itemItemSimilarity)

In [131]:
# Evaluation

In [143]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [144]:
predict = []
for i in range(testDF.shape[0]):
    x = (testDF.userId[i], testDF.Index[i])
    predict.append(recommends[(x[0]-1),x[1]])
testDF['Predicted_Rating'] = predict

In [145]:
testDF

,index,userId,movieId,rating,timestamp,Index,Predicted_Rating
0,40057,292,7438,4.5,1140149094,5141,4.023252
1,15056,98,7438,5.0,1459405098,5141,4.193793
2,92715,615,7438,4.0,1408778718,5141,3.498541
3,3990,22,7438,4.5,1131663403,5141,3.617531
4,37377,268,7438,5.0,1314895350,5141,3.555000
5,78155,544,7438,4.5,1435786267,5141,4.476286
6,94464,624,7438,3.5,1083742391,5141,2.304727
7,14440,94,7438,2.0,1291779607,5141,3.592337
8,48200,353,7438,3.5,1112268544,5141,2.717344
9,16119,103,7438,5.0,1117404267,5141,3.846212
